### Melon 100 Chart 스크래핑
* 100곡의 노래의 제목과 Song ID 파싱
* Song ID로 상세페이지로 100번 요청을 해서 상세정보를 파싱
* 100곡의 상세정보를 json file에 저장
* json file을 표데이터를 만들기
* 표데이터를 DB에 저장

In [1]:
import requests
from bs4 import BeautifulSoup
import re

url = 'https://www.melon.com/chart/index.htm'
req_header = {
    #browser 정보
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36'
}
res = requests.get(url, headers=req_header)
print(res.status_code)
if res.ok:
    soup = BeautifulSoup(res.text, 'html.parser')
    print(len(soup.select("div#tb_list"))) #1개
    print(len(soup.select("div#tb_list tr"))) #101개
    print(len(soup.select("div#tb_list tr div.wrap_song_info"))) #200개
    print(len(soup.select("div#tb_list tr div.wrap_song_info a[href*='playSong']"))) #100개
    
    atag_list = soup.select("div#tb_list tr div.wrap_song_info a[href*='playSong']")
    
    #100곡의 Song 정보를 저장할 List 선언
    song_list = []
    
    for idx,atag in enumerate(atag_list,1):
        #1곡의 Song 정보를 저장할 Dict 선언
        song_dict = {}
        
        title = atag.text
        href = atag['href']
        
        matched = re.search(r'(\d+)\);', href)
        song_id = None
        if matched:
            song_id = matched.group(1)
        
        song_url = f'https://www.melon.com/song/detail.htm?songId={song_id}'
        #print(f'{idx}위, {title} {song_url}')
        
        song_dict['title'] = title    
        song_dict['song_id'] = song_id
        song_dict['song_url'] = song_url
        
        song_list.append(song_dict)

print(len(song_list))
song_list[0:3]

200
1
101
200
100
100


[{'title': 'Savage',
  'song_id': '34041584',
  'song_url': 'https://www.melon.com/song/detail.htm?songId=34041584'},
 {'title': 'STAY',
  'song_id': '33658563',
  'song_url': 'https://www.melon.com/song/detail.htm?songId=33658563'},
 {'title': 'My Universe',
  'song_id': '33998510',
  'song_url': 'https://www.melon.com/song/detail.htm?songId=33998510'}]

In [15]:
import requests
from bs4 import BeautifulSoup
import re

req_header = {
    #browser 정보
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36'
}

song_detail_list = []
for idx,song in enumerate(song_list,1):
    url = song['song_url']
    res = requests.get(url, headers=req_header)
    print(idx, res.status_code, song['title'])
    
    song_detail_dict = {}
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')
        
        song_detail_dict['곡명'] = song['title']
        
        span_tag = soup.select("a[href*='goArtistDetail'] span")
        if span_tag:
            song_detail_dict['가수'] = span_tag[0].text
         
        dd_tag = soup.select("div.meta dd")
        if dd_tag:    
            song_detail_dict['앨범'] = dd_tag[0].text
            song_detail_dict['발매일'] = dd_tag[1].text
            song_detail_dict['장르'] = dd_tag[2].text
        
        song_id = song['song_id']
        like_url = f'https://www.melon.com/commonlike/getSongLike.json?contsIds={song_id}'
        like_res = requests.get(like_url, headers=req_header)
        #print(like_res.status_code)    
        #print(like_res.json()['contsLike'][0]['SUMMCNT'])
        like_cnt = like_res.json()['contsLike'][0]['SUMMCNT']
        song_detail_dict['좋아요'] = like_cnt
        
        div_tag = soup.select('div#d_video_summary')
        if div_tag:
            lyric = div_tag[0].text
            #print(lyric)
        else:
            lyric = ''
        #정규표현식 Pattern 생성
        reg_exp = re.compile(r'[\n\r\t]')
        #Pattern과 일치하는 문자열을 empty string으로 대체해라
        song_detail_dict['가사'] = reg_exp.sub('', lyric.strip())
        
        #1개의 Song정보를 담고 있는 dict를 list에 추가
        song_detail_list.append(song_detail_dict)
        
print(len(song_detail_list))
song_detail_list[0:3]

1 200 Savage
2 200 STAY
3 200 My Universe
4 200 신호등
5 200 낙하 (with 아이유)
6 200 Next Level
7 200 OHAYO MY NIGHT
8 200 Permission to Dance
9 200 바라만 본다
10 200 Butter
11 200 Weekend
12 200 다정히 내 이름을 부르면
13 200 찬란한 계절
14 200 Bad Habits
15 200 Dynamite
16 200 Peaches (Feat. Daniel Caesar & Giveon)
17 200 시간을 거슬러 (낮에 뜨는 달 X 케이윌)
18 200 좋아좋아
19 200 헤픈 우연
20 200 Queendom
21 200 이제 나만 믿어요
22 200 DUMB DUMB
23 200 고백
24 200 가을 타나 봐
25 200 별빛 같은 나의 사랑아
26 200 라일락
27 200 비와 당신
28 200 너를 생각해
29 200 LOCO
30 200 그대라는 사치
31 200 사랑은 언제나 목마르다 (N번째 연애 X 이해리 (다비치))
32 200 HERO
33 200 비가 오는 날엔 (2021)
34 200 롤린 (Rollin')
35 200 Celebrity
36 200 다시 사랑한다면 (김필 Ver.)
37 200 Dun Dun Dance
38 200 색안경 (STEREOTYPE)
39 200 찰나가 영원이 될 때 (The Eternal Moment)
40 200 ASAP
41 200 끝사랑
42 200 Hey Mama (Feat. Nicki Minaj, Bebe Rexha & Afrojack)
43 200 Savage Love (Laxed - Siren Beat) (BTS Remix)
44 200 잊었니
45 200 사이렌 Remix (Feat. UNEDUCATED KID, Paul Blanco)
46 200 Bk Love
47 200 작은 것들을 위한 시 (Boy With Luv) (Feat. Halsey)
48 20

[{'곡명': 'Savage',
  '가수': 'aespa',
  '앨범': 'Savage - The 1st Mini Album',
  '발매일': '2021.10.05',
  '장르': '댄스',
  '좋아요': 51107,
  '가사': "Oh my gosh!Don't you know I’m a Savage?I’m a Killa 너를 깰 ae아직도 가리고 환각을 펼친 너팰라 We Holler두렵지 않아 너 너 Hit you harder날 밀어 넣어 Deep fake on me준비가 안된 무대로몰아넣어 Fake on meGot everybody mock up to me수치를 느끼게 멘탈을 흔들어놔싸늘한 관중 무너져 ae더는 널 못 참아 Say No!두고 봐 난 좀 Savage너의 Dirty 한 Play더는 두고 볼 수 없어나를 무너뜨리고 싶은네 환각들이 점점너를 구축할 이유가 돼I’m a Savage널 부셔 깨 줄게 OhI’m a Savage널 짓밟아 줄게 OhGet me get me nowGet me get me now(Zu Zu Zu Zu)지금 나를 잡아아님 난 더 Savage(Zu Zu Zu Zu)Get me get me nowGet me get me now(Zu Zu Zu Zu)이젠 내가 너를 잡아Now I'm a SavageGimme gimme nowGimme gimme now(Zu Zu Zu Zu)너의 말이 보여네 약점 Algorithm(Zu Zu Zu Zu)김이 김이 나김이 김이 나(Zu Zu Zu Zu)MA ae SYNK 방해 말고꺼져 Savage(Zu Zu Zu Zu)Mmmh Everybody looks at me익숙하잖니양보해 참아야만 돼어른스럽게I'm locked up in the glass난 놀고 싶은데너무 끔찍한 기대그런 환각 틀에 나를 가둬 놔I'm going 광야로 Game in물리쳐 교묘한 이간질And my ae로부터멀어지게 만들회심찬 네 TrickWe gone 광야로 Game in베어버려 내 빛의 검데미지를 입은 네게인정사정 볼

### Song 정보를 담고 있는 List를 Json File로 저장

In [17]:
import json

with open('data/songs.json','w',encoding='utf8') as file:
    json.dump(song_detail_list, file)

### Json File을 Pandas의 DataFrame 객체로 저장

In [20]:
import pandas as pd

song_df = pd.read_json('data/songs.json')
print(type(song_df))
song_df.head()

<class 'pandas.core.frame.DataFrame'>


,곡명,가수,앨범,발매일,장르,좋아요,가사
0,Savage,aespa,Savage - The 1st Mini Album,2021.10.05,댄스,51107,Oh my gosh!Don't you know I’m a Savage?I’m a K...
1,STAY,The Kid LAROI,Stay,2021.07.09,POP,165140,I do the same thing I told youthat I never wou...
2,My Universe,Coldplay,My Universe,2021.09.24,POP,104769,"You, you are my universe andI just want to put..."
3,신호등,이무진,신호등,2021.05.14,록/메탈,222188,이제야 목적지를 정했지만가려한 날 막아서네 난 갈 길이 먼데새빨간 얼굴로 화를 냈던...
4,낙하 (with 아이유),AKMU (악뮤),NEXT EPISODE,2021.07.26,록/메탈,133812,"말했잖아 언젠가이런 날이 온다면난 널 혼자 내버려두지 않을 거라고 죄다 낭떠러지야,..."


In [22]:
song_df.tail()

,곡명,가수,앨범,발매일,장르,좋아요,가사
95,너 하나야 (낮에 뜨는 달 X 길구봉구),길구봉구,너 하나야 (낮에 뜨는 달 X 길구봉구),2021.09.17,발라드,9711,내 가슴이 뛰고 있는걸 알려준 사람숨을 쉴 때 마다 차오르는 한 사람밀어내려고 해봐...
96,GLASSY,조유리,GLASSY,2021.10.07,댄스,9376,기분이 들떠 Like a star like a star 걸음에 시선이 쏟아져 아닌척...
97,여전히 아름다운지,세븐틴,슬기로운 의사생활 시즌2 OST Part 8,2021.08.13,"발라드, 국내드라마",45209,첨엔 혼자라는 게 편했지자유로운 선택과 시간에너의 기억을 지운 듯 했어정말 난 그런...
98,Dolphin,오마이걸 (OH MY GIRL),NONSTOP,2020.04.27,댄스,192444,oh my god 타이밍이 참 얄미워오늘 같은 날 마주쳐 이게 뭐야머리는 엉망인 데...
99,12:45 (Stripped),Etham,12:45 (Stripped),2018.10.26,POP,151390,It's 12 45 on a Tuesdayand I don't really care...


### song_detail_list를 DataFrame으로 저장하기
* song_detail_dict <=> Series
* song_detail_list <=> DataFrame

In [25]:
import pandas as pd

#empty DataFrame 객체생성
song_df2 = pd.DataFrame(columns=['곡명','가수','앨범','발매일','장르','좋아요','가사'])
for song_dict in song_detail_list:
    song_series = pd.Series(song_dict)
    song_df2 = song_df2.append(song_series, ignore_index=True)

song_df2.head()

,곡명,가수,앨범,발매일,장르,좋아요,가사
0,Savage,aespa,Savage - The 1st Mini Album,2021.10.05,댄스,51107,Oh my gosh!Don't you know I’m a Savage?I’m a K...
1,STAY,The Kid LAROI,Stay,2021.07.09,POP,165140,I do the same thing I told youthat I never wou...
2,My Universe,Coldplay,My Universe,2021.09.24,POP,104769,"You, you are my universe andI just want to put..."
3,신호등,이무진,신호등,2021.05.14,록/메탈,222188,이제야 목적지를 정했지만가려한 날 막아서네 난 갈 길이 먼데새빨간 얼굴로 화를 냈던...
4,낙하 (with 아이유),AKMU (악뮤),NEXT EPISODE,2021.07.26,록/메탈,133812,"말했잖아 언젠가이런 날이 온다면난 널 혼자 내버려두지 않을 거라고 죄다 낭떠러지야,..."


In [30]:
print('shape ', song_df.shape)
print('columns ', song_df.columns)
print('index ', song_df.index)
print('values 타입', type(song_df.values))

shape  (100, 7)
columns  Index(['곡명', '가수', '앨범', '발매일', '장르', '좋아요', '가사'], dtype='object')
index  RangeIndex(start=0, stop=100, step=1)
values 타입 <class 'numpy.ndarray'>


In [34]:
song_df.values[0]

array(['Savage', 'aespa', 'Savage - The 1st Mini Album', '2021.10.05',
       '댄스', 51107,
       "Oh my gosh!Don't you know I’m a Savage?I’m a Killa 너를 깰 ae아직도 가리고 환각을 펼친 너팰라 We Holler두렵지 않아 너 너 Hit you harder날 밀어 넣어 Deep fake on me준비가 안된 무대로몰아넣어 Fake on meGot everybody mock up to me수치를 느끼게 멘탈을 흔들어놔싸늘한 관중 무너져 ae더는 널 못 참아 Say No!두고 봐 난 좀 Savage너의 Dirty 한 Play더는 두고 볼 수 없어나를 무너뜨리고 싶은네 환각들이 점점너를 구축할 이유가 돼I’m a Savage널 부셔 깨 줄게 OhI’m a Savage널 짓밟아 줄게 OhGet me get me nowGet me get me now(Zu Zu Zu Zu)지금 나를 잡아아님 난 더 Savage(Zu Zu Zu Zu)Get me get me nowGet me get me now(Zu Zu Zu Zu)이젠 내가 너를 잡아Now I'm a SavageGimme gimme nowGimme gimme now(Zu Zu Zu Zu)너의 말이 보여네 약점 Algorithm(Zu Zu Zu Zu)김이 김이 나김이 김이 나(Zu Zu Zu Zu)MA ae SYNK 방해 말고꺼져 Savage(Zu Zu Zu Zu)Mmmh Everybody looks at me익숙하잖니양보해 참아야만 돼어른스럽게I'm locked up in the glass난 놀고 싶은데너무 끔찍한 기대그런 환각 틀에 나를 가둬 놔I'm going 광야로 Game in물리쳐 교묘한 이간질And my ae로부터멀어지게 만들회심찬 네 TrickWe gone 광야로 Game in베어버려 내 빛의 검데미지를 입은 네게인정사정 볼 것 없는 펀치그것 봐 난 좀 Savage너의 재생력을 막아흐트러놔

In [36]:
song_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   곡명      100 non-null    object
 1   가수      100 non-null    object
 2   앨범      100 non-null    object
 3   발매일     100 non-null    object
 4   장르      100 non-null    object
 5   좋아요     100 non-null    int64 
 6   가사      100 non-null    object
dtypes: int64(1), object(6)
memory usage: 5.6+ KB


In [39]:
#1개의 컬럼 선택
print(type(song_df['가수']))
song_df['가수']

<class 'pandas.core.series.Series'>


0                 aespa
1         The Kid LAROI
2              Coldplay
3                   이무진
4             AKMU (악뮤)
            ...        
95                 길구봉구
96                  조유리
97                  세븐틴
98    오마이걸 (OH MY GIRL)
99                Etham
Name: 가수, Length: 100, dtype: object

In [41]:
#컬럼의 값별로 row counting
song_df['가수'].value_counts()

임영웅             12
방탄소년단            6
아이유              6
폴킴               3
이무진              3
                ..
스탠딩 에그           1
TWICE (트와이스)     1
HYNN (박혜원)       1
Coldplay         1
ITZY (있지)        1
Name: 가수, Length: 62, dtype: int64

In [43]:
#unique한 컬럼의 값
song_df['장르'].unique()

array(['댄스', 'POP', '록/메탈', '랩/힙합', '발라드', 'R&B/Soul', '발라드, 국내드라마',
       '성인가요', '발라드, 인디음악', '포크/블루스', '록/메탈, 국내드라마', 'R&B/Soul, 인디음악',
       '일렉트로니카', '인디음악, 포크/블루스'], dtype=object)

In [45]:
#컬럼의 값이 empty string check
song_df.loc[song_df['가사'] == '']

,곡명,가수,앨범,발매일,장르,좋아요,가사
41,"Hey Mama (Feat. Nicki Minaj, Bebe Rexha & Afro...",David Guetta,Listen,2014.11.25,일렉트로니카,44187,


In [47]:
#lyric이 없는 것을 확인
song_list[40:42]

[{'title': '끝사랑',
  'song_id': '33630420',
  'song_url': 'https://www.melon.com/song/detail.htm?songId=33630420'},
 {'title': 'Hey Mama (Feat. Nicki Minaj, Bebe Rexha & Afrojack)',
  'song_id': '5473462',
  'song_url': 'https://www.melon.com/song/detail.htm?songId=5473462'}]

### 특정행과 열을 선택(Selection)
* loc[] 또는 iloc[] 사용
* Slicing 사용해서 구간을 선택
* [] 사용해서 특정행과 특정열을 선택
* 조건식을 사용해서 조건을 만족하는 행을 선택

In [51]:
# song_df.loc[0:5,:]
song_df.loc[0:3]

,곡명,가수,앨범,발매일,장르,좋아요,가사
0,Savage,aespa,Savage - The 1st Mini Album,2021.10.05,댄스,51107,Oh my gosh!Don't you know I’m a Savage?I’m a K...
1,STAY,The Kid LAROI,Stay,2021.07.09,POP,165140,I do the same thing I told youthat I never wou...
2,My Universe,Coldplay,My Universe,2021.09.24,POP,104769,"You, you are my universe andI just want to put..."
3,신호등,이무진,신호등,2021.05.14,록/메탈,222188,이제야 목적지를 정했지만가려한 날 막아서네 난 갈 길이 먼데새빨간 얼굴로 화를 냈던...


In [54]:
#loc은 컬럼명을 slicing
song_df.loc[0:3,'곡명':'앨범']

,곡명,가수,앨범
0,Savage,aespa,Savage - The 1st Mini Album
1,STAY,The Kid LAROI,Stay
2,My Universe,Coldplay,My Universe
3,신호등,이무진,신호등


In [57]:
#iloc은 인덱스를 slicing
song_df.iloc[0:3,0:4]

,곡명,가수,앨범,발매일
0,Savage,aespa,Savage - The 1st Mini Album,2021.10.05
1,STAY,The Kid LAROI,Stay,2021.07.09
2,My Universe,Coldplay,My Universe,2021.09.24


In [59]:
#1개씩 skip
song_df.loc[0:9:2,'가수':'좋아요']

,가수,앨범,발매일,장르,좋아요
0,aespa,Savage - The 1st Mini Album,2021.10.05,댄스,51107
2,Coldplay,My Universe,2021.09.24,POP,104769
4,AKMU (악뮤),NEXT EPISODE,2021.07.26,록/메탈,133812
6,디핵 (D-Hack),OHAYO MY NIGHT,2020.06.20,랩/힙합,141510
8,MSG워너비(M.O.M),MSG워너비 1집,2021.06.26,발라드,159146


In [66]:
print(type(song_df.loc[[3,7,10],'가수']))
song_df.loc[[3,7,10],'가수']

<class 'pandas.core.series.Series'>


3              이무진
7            방탄소년단
10    태연 (TAEYEON)
Name: 가수, dtype: object

In [68]:
print(type(song_df.loc[[3,7,10],['가수']]))
song_df.loc[[3,7,10],['가수']]

<class 'pandas.core.frame.DataFrame'>


,가수
3,이무진
7,방탄소년단
10,태연 (TAEYEON)


In [71]:
song_df.loc[0:3,['곡명','가수','장르','발매일']]

,곡명,가수,장르,발매일
0,Savage,aespa,댄스,2021.10.05
1,STAY,The Kid LAROI,POP,2021.07.09
2,My Universe,Coldplay,POP,2021.09.24
3,신호등,이무진,록/메탈,2021.05.14


In [74]:
song_df.loc[song_df['가수'] == '방탄소년단','곡명':'좋아요']

,곡명,가수,앨범,발매일,장르,좋아요
7,Permission to Dance,방탄소년단,Butter / Permission to Dance,2021.07.09,댄스,180119
9,Butter,방탄소년단,Butter,2021.05.21,댄스,246028
14,Dynamite,방탄소년단,Dynamite (DayTime Version),2020.08.24,댄스,424630
46,작은 것들을 위한 시 (Boy With Luv) (Feat. Halsey),방탄소년단,MAP OF THE SOUL : PERSONA,2019.04.12,댄스,429585
51,봄날,방탄소년단,YOU NEVER WALK ALONE,2017.02.13,랩/힙합,549818
55,Life Goes On,방탄소년단,BE,2020.11.20,랩/힙합,227769


In [79]:
#장르가 댄스인 Song의 곡명,가수,발매일 선택하고 index를 reset
song_df.loc[song_df['장르'] == '댄스',['곡명','가수','장르','발매일']].reset_index(drop=True)

,곡명,가수,장르,발매일
0,Savage,aespa,댄스,2021.10.05
1,Next Level,aespa,댄스,2021.05.17
2,Permission to Dance,방탄소년단,댄스,2021.07.09
3,Butter,방탄소년단,댄스,2021.05.21
4,Weekend,태연 (TAEYEON),댄스,2021.07.06
5,Dynamite,방탄소년단,댄스,2020.08.24
6,Queendom,Red Velvet (레드벨벳),댄스,2021.08.16
7,DUMB DUMB,전소미,댄스,2021.08.02
8,라일락,아이유,댄스,2021.03.25
9,LOCO,ITZY (있지),댄스,2021.09.24


In [86]:
#장르가 댄스인 Song의 곡명,가수,발매일 선택하고 최근 발매일 순서로 정렬
dance_df = song_df.loc[song_df['장르'] == '댄스',['곡명','가수','장르','발매일']]
#정렬하고 index를 reset 한 결과를 저장
dance_df = dance_df.sort_values(by='발매일', ascending=False).reset_index()
#컬럼명을 변경
dance_df.columns = ['순위','곡명','가수','장르','발매일']

In [90]:
dance_df.head(10)

,순위,곡명,가수,장르,발매일
0,96,GLASSY,조유리,댄스,2021.10.07
1,0,Savage,aespa,댄스,2021.10.05
2,28,LOCO,ITZY (있지),댄스,2021.09.24
3,37,색안경 (STEREOTYPE),STAYC(스테이씨),댄스,2021.09.06
4,19,Queendom,Red Velvet (레드벨벳),댄스,2021.08.16
5,21,DUMB DUMB,전소미,댄스,2021.08.02
6,7,Permission to Dance,방탄소년단,댄스,2021.07.09
7,10,Weekend,태연 (TAEYEON),댄스,2021.07.06
8,65,치맛바람 (Chi Mat Ba Ram),브레이브걸스,댄스,2021.06.17
9,66,Alcohol-Free,TWICE (트와이스),댄스,2021.06.09
